In [16]:
import numpy as np
from qiskit.circuit.library import UnitaryGate
from qiskit import QuantumCircuit, transpile
from quantum_algorithms import QuantumPhaseEstimation
from qiskit_aer import Aer

### $\text{Unitary Operator for Validation Test}$

In [ ]:
# Pauli-Z in matrix form
unitary_matrix = np.array([
    [1, 0],
    [0, -1]
], dtype=complex)

### $\text{Classical Approach}$

In [ ]:
# Determine the eigenvalues and eigenvectors using NumPy
eigenvalues_exact, eigenvectors_exact = np.linalg.eig(unitary_matrix)

# Logging
for val, vec in zip(eigenvalues_exact, eigenvectors_exact):
    print(f"""Eigenvector: {vec} ===> Eigenvalue: {val:.4f}""") 

Eigenvector: [1. 0.] ===> Eigenvalue: 1.0
Eigenvector: [0. 1.] ===> Eigenvalue: -1.0


### $\text{Quantum Approach}$

In [15]:
# Qubit initialization
n_control = 2
n_target = 1
n_total = n_control + n_target

# Prepare the unitary operator
unitary_gate = UnitaryGate(unitary_matrix, label="U")

# Build a quantum circuit
qc = QuantumCircuit(n_total, n_control)

# Initialize the eigenstate |1>
qc.x(n_total - 1)

# Apply the Quantum Phase Estimation
QuantumPhaseEstimation(qc, unitary_gate, n_control, n_target)

# Measurement
for order in range(n_control):
    qc.measure(order, order)

# Draw the circuit
qc.draw()

┌───┐                ░     ░ ┌───┐               ░ ┌─┐   
q_0: ┤ H ├──■─────────────░──X──░─┤ H ├─■─────────────░─┤M├───
     ├───┤  │             ░  │  ░ └───┘ │P(π/2) ┌───┐ ░ └╥┘┌─┐
q_1: ┤ H ├──┼────■────■───░──X──░───────■───────┤ H ├─░──╫─┤M├
     ├───┤┌─┴─┐┌─┴─┐┌─┴─┐ ░     ░               └───┘ ░  ║ └╥┘
q_2: ┤ X ├┤ U ├┤ U ├┤ U ├─░─────░─────────────────────░──╫──╫─
     └───┘└───┘└───┘└───┘ ░     ░                     ░  ║  ║ 
c: 2/════════════════════════════════════════════════════╩══╩═
                                                         0  1

In [17]:
# Simulation
simulator = Aer.get_backend("qasm_simulator")
compiled = transpile(qc, simulator)
job = simulator.run(compiled, shots=1000)
result = job.result()

# Show the result of simulation
counts = result.get_counts()
print(counts)

{'10': 1000}


In [ ]:
# Logging
theta = int(str(max(counts)), 2) / 2**(n_control)
power = 2 * np.pi * theta
eigenvalues_predict = complex(np.cos(power), np.sin(power))

print(f"Eigenstate: [0. 1.] ===> Eigenvalue: {eigenvalues_predict:.4f}")

Eigenstate: [0. 1.] ===> Eigenvalue: -1.0000+0.0000j
